# CH. 7 - TOPIC MODELS
## Activities

#### Activity 7.01

In [1]:
# not necessary
# added to suppress warnings coming from pyLDAvis

import warnings
warnings.filterwarnings('ignore')

In [2]:
import langdetect  # language detection
import matplotlib.pyplot  # plotting
import nltk  # natural language processing
import numpy  # arrays and matrices
import pandas  # dataframes
import pyLDAvis  # plotting
import pyLDAvis.sklearn  # plotting
import regex  # regular expressions
import sklearn  # machine learning

In [3]:
# define path

path = 'latimeshealth.txt'

In [4]:
# load data

df = pandas.read_csv(path, sep="|", header=None)
df.columns = ["id", "datetime", "tweettext"]

In [5]:
# define quick look function for data frame

def dataframe_quick_look(df, nrows):
    print("SHAPE:\n{shape}\n".format(shape=df.shape))
    print("COLUMN NAMES:\n{names}\n".format(names=df.columns))
    print("HEAD:\n{head}\n".format(head=df.head(nrows)))

In [6]:
dataframe_quick_look(df, nrows=2)

SHAPE:
(4171, 3)

COLUMN NAMES:
Index(['id', 'datetime', 'tweettext'], dtype='object')

HEAD:
                   id                        datetime  \
0  576760256031682561  Sat Mar 14 15:02:15 +0000 2015   
1  576715414811471872  Sat Mar 14 12:04:04 +0000 2015   

                                           tweettext  
0  Five new running shoes that aim to go the extr...  
1  Gym Rat: Disq class at Crunch is intense worko...  



In [7]:
# view final data that will be carried forward

raw = df['tweettext'].tolist()
print("HEADLINES:\n{lines}\n".format(lines=raw[:5]))
print("LENGTH:\n{length}\n".format(length=len(raw)))

HEADLINES:
['Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU', 'Gym Rat: Disq class at Crunch is intense workout on pulley system http://lat.ms/1EKOFdr', 'Noshing through thousands of ideas at Natural Products Expo West http://lat.ms/1EHqywg', 'Natural Products Expo also explores beauty, supplements and more http://lat.ms/1EHqyfE', 'Free Fitness Weekends in South Bay beach cities aim to spark activity http://lat.ms/1EH3SMC']

LENGTH:
4171



In [8]:
# define function for checking language of tweets
# filter to english only

def do_language_identifying(txt):
    try:
       the_language = langdetect.detect(txt)
    except:
       the_language = 'none'
    return the_language

In [9]:
# define function to perform lemmatization

def do_lemmatizing(wrd):
    out = nltk.corpus.wordnet.morphy(wrd)
    return (wrd if out is None else out)

In [10]:
# define function to cleaning tweet data

def do_tweet_cleaning(txt):
    # identify language of tweet
    # return null if language not english
    lg = do_language_identifying(txt)
    if lg != 'en':
        return None
    
    # split the string on whitespace
    out = txt.split(' ')
    
    # identify screen names
    # replace with SCREENNAME
    out = ['SCREENNAME' if i.startswith('@') else i for i in out]
    
    # identify urls
    # replace with URL
    out = [
        'URL' if bool(regex.search('http[s]?://', i)) 
        else i for i in out
    ]
    
    # remove all punctuation
    out = [regex.sub('[^\\w\\s]|\n', '', i) for i in out]
    
    # make all non-keywords lowercase
    keys = ['SCREENNAME', 'URL']
    out = [i.lower() if i not in keys else i for i in out]
    
    # remove keywords
    out = [i for i in out if i not in keys]
    
    # remove stopwords
    list_stop_words = nltk.corpus.stopwords.words('english')
    list_stop_words = [regex.sub('[^\\w\\s]', '', i) for i in list_stop_words]
    
    out = [i for i in out if i not in list_stop_words]
    
    # lemmatizing
    out = [do_lemmatizing(i) for i in out]
    
    # keep words 4 or more characters long
    out = [i for i in out if len(i) >= 5]
    
    return out

In [11]:
# apply cleaning function to every tweet

clean = list(map(do_tweet_cleaning, raw))

In [13]:
# remove none types

clean = list(filter(None.__ne__, clean))
print("HEADLINES:\n{lines}\n".format(lines=clean[:5]))
print("LENGTH:\n{length}\n".format(length=len(clean)))

HEADLINES:
[['running', 'shoes', 'extra'], ['class', 'crunch', 'intense', 'workout', 'pulley', 'system'], ['thousand', 'natural', 'product'], ['natural', 'product', 'explore', 'beauty', 'supplement'], ['fitness', 'weekend', 'south', 'beach', 'spark', 'activity']]

LENGTH:
4084



In [14]:
# turn tokens back into strings
# concatenate using whitespaces

clean_sentences = [" ".join(i) for i in clean]

In [15]:
print(clean_sentences[0:10])

['running shoes extra', 'class crunch intense workout pulley system', 'thousand natural product', 'natural product explore beauty supplement', 'fitness weekend south beach spark activity', 'kayla harrison sacrifice', 'sonic treatment alzheimers disease', 'ultrasound brain restore memory alzheimers needle onlyso farin mouse', 'apple researchkit really medical research', 'warning chantix drink taking might remember']


#### Activity 7.02

In [16]:
# define global variables

number_words = 10
number_docs = 10
number_features = 1000

In [17]:
# bag of words conversion
# count vectorizer (raw counts)

vectorizer1 = sklearn.feature_extraction.text.CountVectorizer(
    analyzer="word",
    max_df=0.95, 
    min_df=10, 
    max_features=number_features
)
clean_vec1 = vectorizer1.fit_transform(clean_sentences)
print(clean_vec1[0])

feature_names_vec1 = vectorizer1.get_feature_names()

  (0, 319)	1


In [18]:
# define function to calculate perplexity based on number of topics

def perplexity_by_ntopic(data, ntopics):
    output_dict = {
        "Number Of Topics": [], 
        "Perplexity Score": []
    }
    
    for t in ntopics:
        lda = sklearn.decomposition.LatentDirichletAllocation(
            n_components=t,
            learning_method="online",
            random_state=0
        )
        lda.fit(data)
        
        output_dict["Number Of Topics"].append(t)
        output_dict["Perplexity Score"].append(lda.perplexity(data))
        
    output_df = pandas.DataFrame(output_dict)
    
    index_min_perplexity = output_df["Perplexity Score"].idxmin()
    output_num_topics = output_df.loc[
        index_min_perplexity,  # index
        "Number Of Topics"  # column
    ]
        
    return (output_df, output_num_topics)

In [19]:
# execute function on vector of numbers of topics
# takes several minutes

df_perplexity, optimal_num_topics = perplexity_by_ntopic(
    clean_vec1, 
    ntopics=[i for i in range(1, 21) if i % 2 == 0]
)

In [20]:
print(df_perplexity)

   Number Of Topics  Perplexity Score
0                 2        340.783397
1                 4        396.528955
2                 6        426.758275
3                 8        456.847605
4                10        462.031701
5                12        479.395070
6                14        500.844794
7                16        508.523682
8                18        526.037458
9                20        535.933416


In [21]:
# define and fit lda model

lda = sklearn.decomposition.LatentDirichletAllocation(
    n_components=optimal_num_topics,
    learning_method="online",
    random_state=0
)
lda.fit(clean_vec1)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [22]:
# define function to format raw output into nice tables

def get_topics(mod, vec, names, docs, ndocs, nwords):
    # word to topic matrix
    W = mod.components_
    W_norm = W / W.sum(axis=1)[:, numpy.newaxis]
    # topic to document matrix
    H = mod.transform(vec)
    
    W_dict = {}
    H_dict = {}
    
    for tpc_idx, tpc_val in enumerate(W_norm):
        topic = "Topic{}".format(tpc_idx)
        
        # formatting w
        W_indices = tpc_val.argsort()[::-1][:nwords]
        W_names_values = [
            (round(tpc_val[j], 4), names[j]) 
            for j in W_indices
        ]
        W_dict[topic] = W_names_values
        
        # formatting h
        H_indices = H[:, tpc_idx].argsort()[::-1][:ndocs]
        H_names_values = [
            (round(H[:, tpc_idx][j], 4), docs[j]) 
            for j in H_indices
        ]
        H_dict[topic] = H_names_values
        
    W_df = pandas.DataFrame(
        W_dict, 
        index=["Word" + str(i) for i in range(nwords)]
    )
    H_df = pandas.DataFrame(
        H_dict,
        index=["Doc" + str(i) for i in range(ndocs)]
    )
        
    return (W_df, H_df)

In [23]:
# get nice tables

W_df, H_df = get_topics(
    mod=lda,
    vec=clean_vec1,
    names=feature_names_vec1,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [24]:
# word-topic table

print(W_df)

                  Topic0                Topic1
Word0    (0.0764, study)      (0.0455, latfit)
Word1   (0.0285, cancer)      (0.0367, health)
Word2   (0.0222, people)  (0.0163, researcher)
Word3    (0.02, patient)  (0.0156, california)
Word4    (0.0187, could)     (0.015, american)
Word5    (0.0177, brain)      (0.0124, expert)
Word6    (0.0156, death)        (0.012, years)
Word7   (0.0156, doctor)      (0.0116, report)
Word8    (0.0154, heart)    (0.0108, exercise)
Word9  (0.0138, disease)       (0.0099, today)


In [25]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.9498, Here's a depressing new statistic: "G...   
Doc1  (0.9477, New Neurology study confirms that mom...   
Doc2  (0.9443, RT @MyLastBite: 6/14/13 #Fitbit #Pedo...   
Doc3  (0.9443, Look around at 1 and 2-yr-olds. About...   
Doc4  (0.9443, Little innovations make a big differe...   
Doc5  (0.944, When states implement anti-smoking pol...   
Doc6  (0.9425, 74% of doctors surveyed said they hav...   
Doc7  (0.9415, New test from @CityofHope suggests Pa...   
Doc8  (0.9387, In these paralyzed patients, electrod...   
Doc9  (0.9378, Should YOU take aspirin to prevent a ...   

                                                 Topic1  
Doc0  (0.9374, Your ears may be older (or younger) t...  
Doc1  (0.9373, RT @kurtissl: #10000steps #LATFIT - J...  
Doc2  (0.9362, RT @DavanMaharaj: The Western sun ris...  
Doc3  (0.9326, Ready for a serious #workout? Try the...  
Doc4  (0.931, RT @latimesscience: Are you connected ...  
Do

In [26]:
# iteractive plot
# pca biplot and histogram

lda_plot = pyLDAvis.sklearn.prepare(lda, clean_vec1, vectorizer1, R=10)
pyLDAvis.display(lda_plot)

#### Activity 7.03

In [27]:
# bag of words conversion
# tf-idf method

vectorizer2 = sklearn.feature_extraction.text.TfidfVectorizer(
    analyzer="word",
    max_df=0.5, 
    min_df=20, 
    max_features=number_features,
    smooth_idf=False
)
clean_vec2 = vectorizer2.fit_transform(clean_sentences)
print(clean_vec2[0])

feature_names_vec2 = vectorizer2.get_feature_names()

In [28]:
# define and fit nmf model

nmf = sklearn.decomposition.NMF(
    n_components=optimal_num_topics,
    init="nndsvda",
    solver="mu",
    beta_loss="frobenius",
    random_state=0, 
    alpha=0.1, 
    l1_ratio=0.5
)
nmf.fit(clean_vec2)

NMF(alpha=0.1, beta_loss='frobenius', init='nndsvda', l1_ratio=0.5,
    max_iter=200, n_components=2, random_state=0, shuffle=False, solver='mu',
    tol=0.0001, verbose=0)

In [29]:
# get nicely formatted result tables

W_df, H_df = get_topics(
    mod=nmf,
    vec=clean_vec2,
    names=feature_names_vec2,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

In [30]:
# word-topic table

print(W_df)

                  Topic0                Topic1
Word0    (0.3737, study)      (0.5925, latfit)
Word1   (0.0259, cancer)       (0.0484, steps)
Word2   (0.0208, people)       (0.0447, today)
Word3  (0.0187, suggest)    (0.0403, exercise)
Word4   (0.0185, health)  (0.0273, healthtips)
Word5  (0.0184, obesity)     (0.0257, workout)
Word6    (0.0182, brain)     (0.0221, fitness)
Word7   (0.0168, weight)     (0.0203, getting)
Word8    (0.0154, woman)       (0.0143, great)
Word9    (0.0131, death)     (0.0132, morning)


In [31]:
# document-topic table

print(H_df)

                                                 Topic0  \
Doc0  (0.203, Kerosene wick lamps produce both light...   
Doc1  (0.203, One more new gas price record in Calif...   
Doc2  (0.203, Docs' intensive 'get healthy' program ...   
Doc3  (0.203, Texas judge confirms brain death is de...   
Doc4  (0.203, Eggs made from stem cells could treat ...   
Doc5  (0.203, Smartphone apps keep pace with costly ...   
Doc6  (0.203, Therapeutic cloning was going to be th...   
Doc7  (0.203, Jumping lunges bring a burn when time ...   
Doc8  (0.203, Girls who play with Doctor Barbie, Ast...   
Doc9  (0.203, Child's psychological mistreatment may...   

                                                 Topic1  
Doc0  (0.2274, Another vaccine fails to prevent stap...  
Doc1  (0.2274, Happy Friday! Let go of the work week...  
Doc2  (0.2274, Plank position is a great way to tigh...  
Doc3  (0.2274, Healthy pets could lead to healthier ...  
Doc4  (0.2274, RT @LATkarenkaplan: Parents, resist t...  
Do